In [10]:
import requests
import json
import base64
import pandas as pd
import matplotlib as mt

In [11]:
#Requesting access to the spotify web API
client_id = '16fd6188f83242288e8af5e299bc66dd'
client_secret = '6abc3f8f1a714265a17ad6423066ec43'

auth_header_b64 = base64.b64encode(f'{client_id}:{client_secret}'.encode('ascii'))
auth_header = auth_header_b64.decode('ascii')

req_header =  {'Authorization' : f'Basic {auth_header}'}
payload = {"grant_type": "client_credentials"}
url = "https://accounts.spotify.com/api/token"
resp = requests.post(url, headers = req_header, data = payload)
token = resp.json()

token['access_token']

'BQAyJawbAukR-1p3ImkrQSLJI6wlsRFSERTcfZJeiYcTVcbfmRVMHJjc3nf5ICbHcsoGi6MWGfPs1pu9bVM'

In [12]:
#Defining header with access token
header = {
"Accept": "application/json",
"Content-Type": "application/json",
"Authorization": f"Bearer {token['access_token']}"
}

In [13]:
#Get all ablbums from artist and return data frame with list of album ids and names
def albums(artist):
    oi = requests.get(f"https://api.spotify.com/v1/artists/{artist}/albums", headers = header)
    oi = pd.DataFrame(oi.json()['items'])
    oi = oi[oi['album_group'] == 'album']
    oi = oi[['id', 'name', 'release_date']]
    return oi

# oi = albums('2SFL3Vf8W5rnkNhxAfzVZw')
# oi

In [14]:
#Function to get all tracks from one album
def tracks_albums(album_id):
    penis = requests.get(f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=50", headers = header)
    raw = pd.DataFrame(penis.json()['items'])
    penis = raw[['track_number']]
    penis['track_id'] = raw['id']
    #Guardando o nome do album:
    penis['album_id'] = album_id
    return penis
# penis = tracks_albums('5PfjsKZLI9whAwPSfNgnES')
# penis

In [15]:
#Função para pegar lista de musicas de cada album
def album_to_track(id_artista):
    album_dos_artistas = albums(id_artista)
    tracks = map(tracks_albums, album_dos_artistas['id'])
    oi = map(lambda dt : dt.merge(album_dos_artistas, left_on = 'album_id', right_on = 'id'), tracks)
    return pd.concat(oi).drop(columns = ['id'])

In [16]:
#Cds do meu pai
#album_to_track('2SFL3Vf8W5rnkNhxAfzVZw')
#Cds do pat
#oi = album_to_track('3t58jfUhoMLYVO14XaUFLA')


In [17]:
#Getting global audio analysis for a single track
def audio_analysis(track):
    oi = requests.get(f"https://api.spotify.com/v1/audio-features/{track}", headers = header).json()
    return oi
    oi = pd.DataFrame(oi)
    return oi
#oi = audio_analysis('2o8BOZK5q5VB4PDe8IEZd9')

In [18]:
#Function receives an artist and returns all of his albuns with global analysis for each track within each album.
def describing_tracks(artist):
    tracks_artists = album_to_track(artist)
    tracks = map(audio_analysis, tracks_artists['track_id'])
    tracks = pd.DataFrame(tracks)
    oi = tracks.merge(tracks_artists, left_on = "id", right_on = 'track_id').drop(columns = ['id'])
    return oi.drop(columns = ['analysis_url', 'track_href', 'uri'])

In [12]:
#oi = describing_tracks('2SFL3Vf8W5rnkNhxAfzVZw')
oi2 = describing_tracks('3t58jfUhoMLYVO14XaUFLA')

c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
oi = describing_tracks('5YfW24O2P3ljTAZjBRr4jy')
oi2 = describing_tracks('4HNRaNTXCLtwCaTv5Qo5fc')
oi3 = describing_tracks('5Bn4jYRlfMfbxVwfPDGeL8')
oi4 = describing_tracks('3xnqS3i4fWn0P30qIlnWYV')
oi5 = describing_tracks('2CJ5GDZmO801WYjUTvSAis')
final = pd.concat([oi, oi2, oi3, oi4, oi5])
final.to_csv(r'C:\Users\Lenovo\Desktop\dados.csv')


c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
#Function to receive a data frame and organize output into pairs.
def pairing_songs(dataframe):
    dataframe['pair_track'] = ''*len(dataframe)
    for i in range(len(dataframe)-1):
        if dataframe['name'][i] == dataframe['name'][i+1]:
            dataframe['pair_track'][i] = dataframe['track_number'][i+1]
        else:
            dataframe['pair_track'][i] = 'last_song'
        dataframe['pair_track'][len(dataframe)-1] = 'last_song'
        dataframe['danceability2'] = ''*len(dataframe)
        dataframe['energy2'] = ''*len(dataframe)
        dataframe['key2'] = ''*len(dataframe)
        dataframe['loudness2'] = ''*len(dataframe)
        dataframe['mode2'] = ''*len(dataframe)
        dataframe['speechiness2'] = ''*len(dataframe)
        dataframe['acousticness2'] = ''*len(dataframe)
        dataframe['instrumentalness2'] = ''*len(dataframe)
        dataframe['liveness2'] = ''*len(dataframe)
        dataframe['valence2'] = ''*len(dataframe)
        dataframe['tempo2'] = ''*len(dataframe)
        dataframe['duration_ms2'] = ''*len(dataframe)
        dataframe['time_signature2'] = ''*len(dataframe)
    for i in range(len(dataframe)-1):
        if  dataframe['pair_track'][i] != 'last_song':
            dataframe['danceability2'][i] = dataframe['danceability'][i+1]
            dataframe['energy2'][i] = dataframe['energy'][i+1]
            dataframe['key2'][i] = dataframe['key'][i+1]
            dataframe['loudness2'][i] = dataframe['loudness'][i+1]
            dataframe['mode2'][i] = dataframe['mode'][i+1]
            dataframe['speechiness2'][i] = dataframe['speechiness'][i+1]
            dataframe['acousticness2'][i] = dataframe['acousticness'][i+1]
            dataframe['instrumentalness2'][i] = dataframe['instrumentalness'][i+1]
            dataframe['liveness2'][i] = dataframe['liveness'][i+1]
            dataframe['valence2'][i] = dataframe['valence'][i+1]
            dataframe['tempo2'][i] = dataframe['tempo'][i+1]
            dataframe['duration_ms2'][i] = dataframe['duration_ms'][i+1]
            dataframe['time_signature2'][i] = dataframe['time_signature'][i+1]
        else:
            dataframe['danceability2'][i] = 'no_pair'
            dataframe['energy2'][i] = 'no_pair'
            dataframe['key2'][i] = 'no_pair'
            dataframe['loudness2'][i] = 'no_pair'
            dataframe['mode2'][i] = 'no_pair'
            dataframe['speechiness2'][i] = 'no_pair'
            dataframe['acousticness2'][i] = 'no_pair'
            dataframe['instrumentalness2'][i] = 'no_pair'
            dataframe['liveness2'][i] = 'no_pair'
            dataframe['valence2'][i] = 'no_pair'
            dataframe['tempo2'][i] = 'no_pair'
            dataframe['duration_ms2'][i] = 'no_pair'
            dataframe['time_signature2'][i] = 'no_pair'
        dataframe['danceability2'][len(dataframe)-1] = 'no_pair'
        dataframe['energy2'][len(dataframe)-1] = 'no_pair'
        dataframe['key2'][len(dataframe)-1] = 'no_pair'
        dataframe['loudness2'][len(dataframe)-1] = 'no_pair'
        dataframe['mode2'][len(dataframe)-1] = 'no_pair'
        dataframe['speechiness2'][len(dataframe)-1] = 'no_pair'
        dataframe['acousticness2'][len(dataframe)-1] = 'no_pair'
        dataframe['instrumentalness2'][len(dataframe)-1] = 'no_pair'
        dataframe['liveness2'][len(dataframe)-1] = 'no_pair'
        dataframe['valence2'][len(dataframe)-1] = 'no_pair'
        dataframe['tempo2'][len(dataframe)-1] = 'no_pair'
        dataframe['duration_ms2'][len(dataframe)-1] = 'no_pair'
        dataframe['time_signature2'][len(dataframe)-1] = 'no_pair'

    return dataframe

#Experimental
pat_ordem = oi
pat_ordem = pairing_songs(pat_ordem)




In [43]:
#Experimental
pat_ordem = oi
pat_ordem = pairing_songs(pat_ordem)
pat_ordem['condition'] = 'experimental'
#Control

pat_shuffle = pat_ordem.sample(frac=1)
pat_shuffle = pairing_songs(pat_shuffle)
pat_shuffle['condition'] = 'control'



c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\lenovo\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [44]:
pat_shuffle

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,mode2,speechiness2,acousticness2,instrumentalness2,liveness2,valence2,tempo2,duration_ms2,time_signature2,condition
39,0.666,0.428,11,-9.274,1,0.0446,0.945,0.923000,0.0883,0.491,...,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,control
51,0.476,0.187,9,-10.300,1,0.0382,0.951,0.000875,0.0941,0.112,...,0,0.039,0.932,0.318,0.0772,0.46,164.38,400480,3,control
60,0.495,0.723,2,-7.790,1,0.0757,0.385,0.830000,0.0827,0.750,...,1,0.0306,0.947,0.0213,0.372,0.0514,98.273,304307,4,control
99,0.662,0.826,0,-5.568,0,0.0411,0.680,0.048800,0.0574,0.963,...,1,0.0374,0.986,0.413,0.0908,0.615,113.651,237933,4,control
74,0.609,0.311,9,-10.165,1,0.0398,0.947,0.188000,0.1270,0.581,...,1,0.0412,0.9,0.624,0.105,0.273,94.311,271813,4,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.630,0.281,4,-9.729,0,0.0421,0.943,0.731000,0.1150,0.433,...,0,0.0474,0.956,0.02,0.0862,0.246,117.277,281520,4,control
72,0.622,0.108,0,-18.240,1,0.0562,0.866,0.006810,0.2310,0.348,...,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,no_pair,control
95,0.553,0.663,0,-8.176,0,0.1570,0.499,0.070000,0.1170,0.512,...,1,0.0695,0.7,0.88,0.0801,0.772,104.052,195733,4,control
15,0.553,0.771,0,-7.596,1,0.2030,0.624,0.117000,0.9870,0.580,...,1,0.0407,0.842,0.611,0.705,0.495,107.218,299600,4,control


In [45]:
final = pd.concat([pat_ordem, pat_shuffle])

In [46]:
final = final[final['mode2'] != 'no_pair']

In [47]:
final.to_csv(r'C:\Users\Lenovo\Desktop\dados.csv')